In [ ]:
import netron
from ultralytics import YOLO

In [ ]:
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6.23M/6.23M [00:04<00:00, 1.41MB/s]



[Adding "Freeze layers" for transfer learning](https://github.com/ultralytics/ultralytics/issues/562)

In [ ]:
model.info(verbose=True)

YOLOv8n summary: 225 layers, 3157200 parameters, 0 gradients, 8.9 GFLOPs


layer                                     name  gradient   parameters                shape         mu      sigma
    0                      model.0.conv.weight     False          432        [16, 3, 3, 3]   -0.00279      0.152
    1                        model.0.bn.weight     False           16                 [16]       2.97       1.86
    2                          model.0.bn.bias     False           16                 [16]      0.249       4.17
    3                      model.1.conv.weight     False         4608       [32, 16, 3, 3]   -0.00012      0.063
    4                        model.1.bn.weight     False           32                 [32]       5.02       1.12
    5                          model.1.bn.bias     False           32                 [32]      0.942        1.5
    6                  model.2.cv1.conv.weight     False         1024       [32, 32, 1, 1]     -0.011     0.0906
    7                    model.2.cv1.bn.weight     False           32                 [32]      

In [ ]:
model.export(format="onnx")

Ultralytics YOLOv8.0.17 🚀 Python-3.10.9 torch-2.0.0.dev20221228 CPU
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from yolov8n.pt with output shape (1, 84, 8400) (6.2 MB)

ONNX: starting export with onnx 1.13.0...
ONNX: export success ✅ 0.3s, saved as yolov8n.onnx (12.2 MB)

Export complete (0.5s)
Results saved to /Users/astadnik/edu/masters/sem_3/CV/5/hairstyle_detector
Predict:         yolo task=detect mode=predict model=yolov8n.onnx -WARNING ⚠️ not yet supported for YOLOv8 exported models
Validate:        yolo task=detect mode=val model=yolov8n.onnx -WARNING ⚠️ not yet supported for YOLOv8 exported models
Visualize:       https://netron.app


========== Diagnostic Run torch.onnx.export version 2.0.0.dev20221228 ==========
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
netron.start("yolov8n.onnx")

Serving 'yolov8n.onnx' at http://localhost:8080


('localhost', 8080)

[FINETUNING TORCHVISION MODELS](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html)